In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from MdpSolverLibs import *
from EnvLibs import TrafficGenerator, getEnvConfig, visualizeEnvConfig, createEnv

In [2]:
configIdx = 0
params = getEnvConfig(configIdx)
visualizeEnvConfig(params)
env = createEnv(params)
env.selectMode(mode="train", type="data")

mdpFormulator = MdpFormulator(params, env.trafficGenerator.getM(mode="train"))
print(np.sum(env.trafficGenerator.getM(mode="train"), axis=1))
print(mdpFormulator.N_states)
print(mdpFormulator.N_actions)
print(mdpFormulator.N_states_original**mdpFormulator.N_user)
print(mdpFormulator.N_states_original**mdpFormulator.N_user*mdpFormulator.N_actions)

Environment Configuration
Number of Users:        4
Window Length:          20
Dataflow:               thumb_fr
Resource Bar:           4
Bandwidth:              40
M List:                 [2, 3]
Random Seed:            999
Alpha Range:            (0.01, 1.0)
Discrete Alpha Steps:   10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.]
256
320
194481
62233920


In [3]:
mdpFormulator.aggregateModel(approximate=True)

In [4]:
mdpKernel = mdpFormulator.getMdpKernel()
print(mdpKernel.rewardTable.shape)
print(mdpKernel.transitionTable.shape)
(V, policy) = mdpKernel.optimize_policy()

(256, 320)
(256, 256, 320)


In [5]:
with open(f'Results/MdpPolicy/mdpKernel_config{configIdx}.pkl', 'wb') as f:
    pickle.dump(mdpKernel, f)   
#with open(f'Results/MdpPolicy/trafficGenerator_{params['dataflow']}_LenWindow{params['LEN_window']}.pkl', 'wb') as f:
#    pickle.dump(trafficGenerator, f)
#print(mdpKernel.V)